### Introduction

This project performs an analysis on data obtained from a supermarket chain in Myanmar. A better understanding of their customer base, such as the Cost Of Goods Sold (COGS) for the different product categories or the product with the highest sales volume, can lead to increased efficiencies and reduced costs.

Examples of the data within the dataset are the ratings of the transactions, the gender of the customer, and the category that the products belonged to.

The dataset was obtained from Kaggle:

https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import warnings
import pygwalker as pyg
import sqlite3
#import sqlalchemy
#from sqlalchemy import create_engine
from sqlalchemy.dialects import sqlite
from pandas.io import sql
import subprocess
import re


warnings.filterwarnings('ignore')

The dataset is in the **CSV** (Comma Seperated Value) format, and will be imported into an **SQLite** database, where it will be preprocessed.

In [2]:
df=pd.read_csv('supermarket.csv')
conn=sqlite3.connect('supermarket.db')
df.to_sql(name='supermarket', con=conn, if_exists='replace', index=False)
conn.commit()

In [3]:
%config SqlMagic.displaylimit = 15

In [4]:
%load_ext sql
%sql sqlite:///supermarket.db

We shall check to make sure that the CSV file has been imported correctly into the SQLite database and that we have managed to successfully connect to it.

In [5]:
%sql select * from supermarket limit 1

Running query in 'sqlite:///supermarket.db'

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1


Next, we shall check to see if there are any duplicate entries. We will be running the check with the invoice ID of the product.

In [6]:
%%sql
SELECT "Invoice ID", COUNT("Invoice ID") AS id_count, SUM(COUNT("Invoice ID")) OVER() AS total_count
FROM supermarket GROUP BY "Invoice ID" having count("Invoice ID")>1

Running query in 'sqlite:///supermarket.db'

Invoice ID,id_count,total_count


It seems that there are no duplicated entries.

In [7]:
#The code that has been commented out can be used to remove duplicated entries

#%%sql

#DELETE FROM supermarket 
#WHERE rowid > (
#  SELECT MIN(rowid) FROM supermarket s  
#  WHERE s."Invoice ID" = supermarket."Invoice ID"
#);


We will also check for null values in the dataset. 

In [8]:
%%sql
SELECT *
FROM supermarket
WHERE ("Invoice ID" or "City" or "Customer type" or "Gender" or "Product line" or "Unit price" or "Quantity" or "Total" or "Date" or "Time" or "Payment" or cogs or "gross margin percentage" or "gross income" or Rating) IS NULL

Running query in 'sqlite:///supermarket.db'

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating


There are no null values in the table.

Finally, we will import the pre-processed SQLite table into Pandas for the Exploratory Data Analysis (EDA)

In [9]:
df = pd.read_sql_query("SELECT * FROM supermarket", conn)
conn.commit()
conn.close()


Let's do a quick check of the dataframe before creating some new columns for some metrics that will be useful for the EDA.

In [10]:
df.describe(include='all')

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
count,1000,1000,1000,1000,1000,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000,1000,1000.00000,1.000000e+03,1000.000000,1000.00000
unique,1000,3,3,2,2,6,NaN,NaN,NaN,NaN,89,506,3,NaN,NaN,NaN,NaN
top,750-67-8428,A,Yangon,Member,Female,Fashion accessories,NaN,NaN,NaN,NaN,2/7/2019,19:48,Ewallet,NaN,NaN,NaN,NaN
freq,1,340,340,501,501,178,NaN,NaN,NaN,NaN,20,7,345,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,55.672130,5.510000,15.379369,322.966749,NaN,NaN,NaN,307.58738,4.761905e+00,15.379369,6.97270
std,NaN,NaN,NaN,NaN,NaN,NaN,26.494628,2.923431,11.708825,245.885335,NaN,NaN,NaN,234.17651,6.131498e-14,11.708825,1.71858
min,NaN,NaN,NaN,NaN,NaN,NaN,10.080000,1.000000,0.508500,10.678500,NaN,NaN,NaN,10.17000,4.761905e+00,0.508500,4.00000
25%,NaN,NaN,NaN,NaN,NaN,NaN,32.875000,3.000000,5.924875,124.422375,NaN,NaN,NaN,118.49750,4.761905e+00,5.924875,5.50000
50%,NaN,NaN,NaN,NaN,NaN,NaN,55.230000,5.000000,12.088000,253.848000,NaN,NaN,NaN,241.76000,4.761905e+00,12.088000,7.00000
75%,NaN,NaN,NaN,NaN,NaN,NaN,77.935000,8.000000,22.445250,471.350250,NaN,NaN,NaN,448.90500,4.761905e+00,22.445250,8.50000


Now we will drop the branch column, as an inspection of the raw dataset revealed that each branch is in a different city, and the name of the city will be more informative than the letter representing the branch.

In [11]:
df.drop(inplace=True, axis=1, columns='Branch')
df.describe(include='all')


,Invoice ID,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
count,1000,1000,1000,1000,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000,1000,1000.00000,1.000000e+03,1000.000000,1000.00000
unique,1000,3,2,2,6,NaN,NaN,NaN,NaN,89,506,3,NaN,NaN,NaN,NaN
top,750-67-8428,Yangon,Member,Female,Fashion accessories,NaN,NaN,NaN,NaN,2/7/2019,19:48,Ewallet,NaN,NaN,NaN,NaN
freq,1,340,501,501,178,NaN,NaN,NaN,NaN,20,7,345,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,55.672130,5.510000,15.379369,322.966749,NaN,NaN,NaN,307.58738,4.761905e+00,15.379369,6.97270
std,NaN,NaN,NaN,NaN,NaN,26.494628,2.923431,11.708825,245.885335,NaN,NaN,NaN,234.17651,6.131498e-14,11.708825,1.71858
min,NaN,NaN,NaN,NaN,NaN,10.080000,1.000000,0.508500,10.678500,NaN,NaN,NaN,10.17000,4.761905e+00,0.508500,4.00000
25%,NaN,NaN,NaN,NaN,NaN,32.875000,3.000000,5.924875,124.422375,NaN,NaN,NaN,118.49750,4.761905e+00,5.924875,5.50000
50%,NaN,NaN,NaN,NaN,NaN,55.230000,5.000000,12.088000,253.848000,NaN,NaN,NaN,241.76000,4.761905e+00,12.088000,7.00000
75%,NaN,NaN,NaN,NaN,NaN,77.935000,8.000000,22.445250,471.350250,NaN,NaN,NaN,448.90500,4.761905e+00,22.445250,8.50000


By looking at the dataframe, it seems that we need to do some feature engineering. For example, the Cost of Goods Sold (cogs) is the total ost of goods sold. By calculating the unit cost of goods sold, it makes it possible to compare between transactions with different quantity of goods sold.

In [12]:
df.head()

,Invoice ID,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


Some feature engineering was also performed to calculate the gross income and Cost Of Goods Sold (COGS) per unit, and to turn the time column into a form that is easier to work with

In [13]:
for i in df:
    df['ucogs']=(df['cogs']/df['Quantity'])
    df['unit gross income'] = (df['gross income']/df['Quantity'])
df.Time.dtypes

dtype('O')

In [14]:
df['Time']=df['Time'].str.replace(':','.')
df['Time'].astype(str).dtypes

dtype('O')

In [15]:
df['hour']=df['Time'].str.split('.', expand=True)[0]
df['minutes']=df['Time'].str.split('.', expand=True)[1]
df.head()

,Invoice ID,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,ucogs,unit gross income,hour,minutes
0,750-67-8428,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13.08,Ewallet,522.83,4.761905,26.1415,9.1,74.69,3.7345,13,08
1,226-31-3081,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10.29,Cash,76.40,4.761905,3.8200,9.6,15.28,0.7640,10,29
2,631-41-3108,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13.23,Credit card,324.31,4.761905,16.2155,7.4,46.33,2.3165,13,23
3,123-19-1176,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20.33,Ewallet,465.76,4.761905,23.2880,8.4,58.22,2.9110,20,33
4,373-73-7910,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10.37,Ewallet,604.17,4.761905,30.2085,5.3,86.31,4.3155,10,37


In [16]:
df['month']=df['Date'].str.split('/',expand=True)[0]
df.head()

,Invoice ID,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,...,Payment,cogs,gross margin percentage,gross income,Rating,ucogs,unit gross income,hour,minutes,month
0,750-67-8428,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,...,Ewallet,522.83,4.761905,26.1415,9.1,74.69,3.7345,13,08,1
1,226-31-3081,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,...,Cash,76.40,4.761905,3.8200,9.6,15.28,0.7640,10,29,3
2,631-41-3108,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,...,Credit card,324.31,4.761905,16.2155,7.4,46.33,2.3165,13,23,3
3,123-19-1176,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,...,Ewallet,465.76,4.761905,23.2880,8.4,58.22,2.9110,20,33,1
4,373-73-7910,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,...,Ewallet,604.17,4.761905,30.2085,5.3,86.31,4.3155,10,37,2


We are ready to start the EDA. For the EDA, we shall use PyGWalker, a high level data visualization tool as it is faster than using Seaborn or Plotly.

In [17]:
walker=pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…